In [ ]:
# # extract text to store to a text file, record the articles whose pdfs or jsons are not available
# input_path = fpath.poten_litera_db
# df = pd.read_csv(input_path, header=None, sep=',')
# df.columns = df_col.db_columns

# # poten_litera_pdf_not_available = fpath.poten_litera_pdf_not_available
# # plib.clear_file(poten_litera_pdf_not_available)

# pdf_folder = fpath.pdf_folder
# text_folder = fpath.text_folder

# for ind in df.index:
#     time.sleep(1)
#     index = int(df.at[ind, "INDEX"])
#     pdf_file_name = str(index) + ".pdf"
#     json_file_name = str(index) + ".json"
#     pdf_path = os.path.join(pdf_folder, pdf_file_name)
#     json_path = os.path.join(pdf_folder, json_file_name)
#     text_path = os.path.join(text_folder, str(index) + ".txt")

#     if os.path.exists(json_path):
#         # pass
#         json2text(json_path, text_path)
#     elif os.path.exists(pdf_path):
#         # pass
#         pdf2text(pdf_path, text_path)
#     else:
#         pass
#         # selected_row = df.iloc[[ind]]
#         # selected_row.to_csv(poten_litera_pdf_not_available, mode='a', header=False, index=False)
#         # print(df.at[ind, "INDEX"], df.at[ind, "DOI"], df.at[ind, "PMID"], df.at[ind, "PMCID"])
#         # print(df.at[ind, "TITLE"])
#         # print(df.at[ind, "FULL_TEXT_URL"], df.at[ind, "FULL_TEXT_SOURCE"])
#         # print(df.at[ind, "PDF_URL"], df.at[ind, "PDF_SOURCE"])
#         # print("\n")

#     print(ind, index)

In [ ]:
# # extract text of length params.text_length_to_extract from the text files and store to a text file
# input_path = fpath.poten_litera_db
# df = pd.read_csv(input_path, header=None, sep=',')
# df.columns = df_col.db_columns

# pdf_folder = fpath.pdf_folder
# text_folder = fpath.text_folder
# processed_text_folder = fpath.processed_texts_of_length_500_folder
# not_recog_articles_folder = fpath.not_recog_articles_folder

# for ind in df.index:
#     # time.sleep(1)
#     index = int(df.at[ind, "INDEX"])
#     pdf_path = os.path.join(pdf_folder, str(index) + ".pdf")
#     json_path = os.path.join(pdf_folder, str(index) + ".json")
#     text_path = os.path.join(text_folder, str(index) + ".txt")
#     text_processed_path = os.path.join(processed_text_folder, str(index) + ".txt")

#     if os.path.exists(text_path):
#         # first we test if the length of the text is greater than the length to extract
#         with open(text_path, 'r', encoding="ascii") as f:
#             text = f.read()
#         f.close()
        
#         text_split = text.split()
        
#         if len(text_split) > params.text_length_to_extract:
#             pass
#             # text_500 = ' '.join(word for word in text_split[:params.text_length_to_extract])
#         else: # if the length of the text is less than the length to extract, we extract the text from the pdf or json once again
#             if os.path.exists(pdf_path):
#                 pdf2text(pdf_path, text_path)
#                 # read from the text file again and check if the length is less than the length to extract
#                 with open(text_path, 'r', encoding="ascii") as f:
#                     text = f.read()
#                 f.close()
#                 text_split = text.split()
#                 if len(text_split) > params.text_length_to_extract:
#                     pass
#                 elif os.path.exists(json_path):
#                     json2text(json_path, text_path)
#                 else:
#                     pass
#             elif os.path.exists(json_path):
#                 json2text(json_path, text_path)
#             else:
#                 raise Exception("No pdf or json file found for index: ", index)

#         # write the text to the processed text file
#         with open(text_path, 'r', encoding="ascii") as f:
#             text = f.read()
#         f.close()
        
#         text_split = text.split()
        
#         if len(text_split) > params.text_length_to_extract:
#             text_500 = ' '.join(word for word in text_split[:params.text_length_to_extract])
#         else:
#             text_500 = text

#             # copy this file to a folder for manual check
#             if os.path.exists(pdf_path):
#                 shutil.copy(pdf_path, not_recog_articles_folder)

#             if os.path.exists(json_path):
#                 shutil.copy(json_path, not_recog_articles_folder)

#             print(df.at[ind, "INDEX"], df.at[ind, "DOI"], df.at[ind, "PMID"], df.at[ind, "PMCID"])
#             print(df.at[ind, "TITLE"])
#             print(df.at[ind, "FULL_TEXT_URL"], df.at[ind, "FULL_TEXT_SOURCE"])
#             print(df.at[ind, "PDF_URL"], df.at[ind, "PDF_SOURCE"])
#             print("\n")
        
#         with open(text_processed_path, 'w', encoding="ascii") as f:
#             f.write(text_500)
#         f.close()
#     else:
#         pass

#     # print(ind, index)

In [ ]:
# count the number of literatures whose pdfs or jsons are not available
input_path = fpath.poten_litera_pdf_not_available
df = pd.read_csv(input_path, header=None, sep=',')
# print(df.shape)
# (406, 11)
# print number of rows
print(df.shape[0])